In [1]:
import datasets

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")

Generating validation split: 100%|██████████| 47728/47728 [00:02<00:00, 23602.51 examples/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [4]:
#document is rand id
#full_text is simply the text
#tokens is full text split by tokens and put into list
#traling white space is a boolean saying whether there is a space after that token
#labels is O if NA or label if it has a label

In [5]:
#up to 29908 are english

In [104]:
document_ids = dataset['train']['id'][:29908]

In [105]:
full_text = dataset['train']['source_text'][:29908]

In [106]:
span_labels = dataset['train']['span_labels'][:29908]

In [141]:
bio_labels = dataset["train"]["mbert_bio_labels"][:29908]

In [107]:
#let's use the spacy english tokenizer
# Construction 2
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

In [108]:
import re

In [109]:
# nline_del_full_text = []
# for sentence in full_text:
#     nline_del_full_text.append(re.sub(r'\\+n', ' ', sentence).replace('\n', ' '))

# full_text = nline_del_full_text

In [114]:
span_labels[0]

'[[440, 453, "USERNAME"], [430, 437, "TIME"], [395, 416, "USERNAME"], [377, 392, "TIME"], [349, 363, "USERNAME"], [344, 346, "TIME"], [321, 330, "USERNAME"], [311, 318, "TIME"], [287, 297, "USERNAME"]]'

In [115]:
tokens = [i.text for i in tokenizer(full_text[0][350: 450])]
whitespaces = [i.whitespace_ for i in tokenizer(full_text[0][350: 450])]

In [116]:
tokens

['ahil.wittauer',
 '-',
 'Meeting',
 'at',
 'quarter',
 'past',
 '13',
 '\n',
 '-',
 'gholamhossein.ruschke',
 '-',
 'Meeting',
 'at',
 '9:47',
 'PM',
 '\n',
 '-',
 'pdmjrsyoz1']

In [117]:
whitespaces

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '',
 '',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '',
 '',
 ' ',
 '']

In [137]:
tokens = [[i.text for i in tokenizer(i)] for i in full_text]

In [130]:
trailing_whitespaces = [[bool(i.whitespace_) for i in tokenizer(i)] for i in full_text]

In [136]:
from ast import literal_eval

label_text = []
labels = set()
for i, spans in enumerate(span_labels):
    spans = literal_eval(spans)
    text = full_text[i]
    for span in spans:
        var_to_repl = text[span[0]:span[1]]
        tokens = [i.text for i in tokenizer(var_to_repl)]
        whitespaces = [i.whitespace_ for i in tokenizer(var_to_repl)]
        if len(tokens) > 1:
            new_tokens = []
            for ind, token in enumerate(tokens):
                if ind == 0:
                    new_tokens.append(f"{span[2]}_START ")
                    #new_tokens.append(whitespaces[ind])
                    labels.add(f"{span[2]}_START")
                elif ind == len(tokens)-1:
                    new_tokens.append(f"{span[2]}_END")
                    new_tokens.append(whitespaces[ind])
                    labels.add(f"{span[2]}_END")
                else:
                    new_tokens.append(f"{span[2]}_MIDDLE ")
                    #new_tokens.append(whitespaces[ind])
                    labels.add(f"{span[2]}_MIDDLE")
            repl = "".join(new_tokens)
            text = text[:span[0]] + repl + text[span[1]:]
        else:
            text = text[:span[0]] + span[2] + text[span[1]:]
            labels.add(span[2])
    label_text.append(text)

In [140]:
labels

{'BOD',
 'BOD_END',
 'BOD_MIDDLE',
 'BOD_START',
 'BUILDING',
 'BUILDING_END',
 'BUILDING_MIDDLE',
 'BUILDING_START',
 'CARDISSUER_END',
 'CARDISSUER_MIDDLE',
 'CARDISSUER_START',
 'CITY',
 'CITY_END',
 'CITY_MIDDLE',
 'CITY_START',
 'COUNTRY',
 'COUNTRY_END',
 'COUNTRY_MIDDLE',
 'COUNTRY_START',
 'DATE',
 'DATE_END',
 'DATE_MIDDLE',
 'DATE_START',
 'DRIVERLICENSE',
 'DRIVERLICENSE_END',
 'DRIVERLICENSE_MIDDLE',
 'DRIVERLICENSE_START',
 'EMAIL',
 'EMAIL_END',
 'EMAIL_MIDDLE',
 'EMAIL_START',
 'GEOCOORD',
 'GEOCOORD_END',
 'GEOCOORD_MIDDLE',
 'GEOCOORD_START',
 'GIVENNAME1',
 'GIVENNAME1_END',
 'GIVENNAME1_MIDDLE',
 'GIVENNAME1_START',
 'GIVENNAME2',
 'GIVENNAME2_END',
 'GIVENNAME2_MIDDLE',
 'GIVENNAME2_START',
 'IDCARD',
 'IDCARD_END',
 'IDCARD_MIDDLE',
 'IDCARD_START',
 'IP',
 'IP_END',
 'IP_MIDDLE',
 'IP_START',
 'LASTNAME1',
 'LASTNAME1_END',
 'LASTNAME1_MIDDLE',
 'LASTNAME1_START',
 'LASTNAME2',
 'LASTNAME2_END',
 'LASTNAME2_MIDDLE',
 'LASTNAME2_START',
 'LASTNAME3',
 'LASTNAME3_EN

In [138]:
label_tokens = [[i.text for i in tokenizer(i)] for i in label_text]

In [139]:
for i, token in enumerate(tokens):
    if len(tokens[i]) != len(label_tokens[i]):
        print(tokens[i])
        print(label_tokens[i])

['<', '?', 'xml', 'version="1.0', '"', 'encoding="UTF-8', '"', '?', '>', '\n', '<', 'online_course', '>', '\n\t', '<', 'course', '>', '\n\t\t', '<', 'instructor', '>', '\n\t\t\t', '<', 'name', '>', 'Ansgar</name', '>', '\n\t\t\t', '<', 'sex', '>', 'M</sex', '>', '\n\t\t\t', '<', 'time>7', "o'clock</time", '>', '\n\t\t', '<', '/instructor', '>', '\n\t\t', '<', 'students', '>', '\n\t\t\t', '<', 'student', '>', '\n\t\t\t\t', '<', 'name', '>', 'Délina</name', '>', '\n\t\t\t\t', '<', 'sex', '>', 'Prefer', 'not', 'to', 'disclose</sex', '>', '\n\t\t\t\t', '<', 'time>3:49am</time', '>', '\n\t\t\t', '<', '/student', '>', '\n\t\t\t', '<', 'student', '>', '\n\t\t\t\t', '<', 'name', '>', 'Szimonetta</name', '>', '\n\t\t\t\t', '<', 'sex', '>', 'F</sex', '>', '\n\t']
['<', '?', 'xml', 'version="1.0', '"', 'encoding="UTF-8', '"', '?', '>', '\n', '<', 'online_course', '>', '\n\t', '<', 'course', '>', '\n\t\t', '<', 'instructor', '>', '\n\t\t\t', '<', 'name', '>', 'GIVENNAME1</name', '>', '\n\t\t\t', '

In [ ]:
len(tokens[0])

87

In [ ]:
for label in label_tokens:
    for i,j in enumerate(label):
        label[i] = 'O'
        for l in labels:
            if l in j:
                label[i] = l
                break


In [ ]:
data = []

for i in range(len(document_ids)):
    data.append({
        'document': document_ids[i][:-1],
        'full_text': full_text[i],
        'tokens': tokens[i],
        'labels': label_tokens[i],
        'trailing_whitespaces': trailing_whitespaces[i]
    })


In [ ]:
import json


with open("train.json", "w") as f:
    f.write(json.dumps(data))